In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix
from contemporary_ner_training.conll_ner_importer import conll_to_ner_labelling  
from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

CONLL NER test:

In [3]:
'''
texts = []
for file in os.listdir('contemporary_ner_training'):
    if file.startswith('estner_split_1'):
        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               
'''

"\ntexts = []\nfor file in os.listdir('contemporary_ner_training'):\n    if file.startswith('estner_split_1'):\n        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               \n"

Flags:

In [3]:
use_vabamorfcorpustagger = False

---

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
def create_training_texts(filenames):
    print("(!) Valmistan ette treenimistekste")
    
    # These files don't work because the protocols are written in a different language,
    # which the goldstandard didn't recognise, hence have no goldstandard tags.
    files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                         'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                         'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                         'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                         'P2rnu_Halliste_Abja_id257_1844a.json', \
                         'Saare_Kaarma_Loona_id7575_1899a.json', \
                         'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                         'J2rva_Tyri_V22tsa_id22178_1912a.json']
    
    start = time.time()
    training_texts = []
    for filename in filenames:
        with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
            if filename in files_not_working:
                continue
            else:
                tagged_text = preprocess_text(json_to_text(file.read()))
                if use_vabamorfcorpustagger:
                    tagged_text.pop_layer('morph_analysis')
                    vm_corpus_tagger.tag([tagged_text])
                training_texts.append(tagged_text)
    print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
    return training_texts

In [6]:
def train_nertagger(training_texts, new_model_dir):
    print("(!) Treenin NerTaggerit")
    start = time.time()
    
    modelUtil = ModelStorageUtil( new_model_dir )
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
    print(f"NerTagger treenitud {time.time() - start} sekundiga.")

In [7]:
def get_testing_and_training_subdistribution(subdistribution):
    training = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            testing = y
        else:
            training.append(y)
    return testing, training

In [8]:
def tag_files(model_dir, testing_files, nertagger, use_vabamorfcorpustagger):
    removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']
    
    print("(!) Märgendan")
    start = time.time()
    
    for test_file in testing_files:
        with open(find(test_file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
            text = f.read()
        
        if test_file == "Tartu_V6nnu_Ahja_id3502_1882a.json":
            text = text.replace('..', '. .')
        text = preprocess_text(Text(text))

        if use_vabamorfcorpustagger or "vabamorf" in model_dir:
            text.pop_layer('morph_analysis')
            text = [text]
            vm_corpus_tagger.tag( text )
            text = text[0]
        nertagger.tag(text)
        text.add_layer(flatten(text['ner'], 'flat_ner'))

        for x in removed_layers:
            text.pop_layer(x)

        path = os.path.join(model_dir, 'vallakohtufailid-trained-nertagger')
        if not os.path.exists(path):
            os.mkdir(path)
            
        text_to_json(text, file=os.path.join(model_dir, 'vallakohtufailid-trained-nertagger', test_file))

            
        print(f'Märgendatud fail {test_file}')
    print(f"Failid märgendatud {time.time() - start} sekundiga.")

In [9]:
def train_model(model_directory):
    for subdistribution in [1, 2, 3, 4, 5]:
        testing, training = get_testing_and_training_subdistribution(subdistribution)
        
        # Get the filenames to be trained on from the files dictionary
        filenames = [key for key, value in files.items() if int(value) in training]

        # Create training_texts from the aforementioned filenames
        #training_texts = create_training_texts(filenames)
        
        # Set up the trainer and training
        new_model_dir = os.path.join('models', model_directory)
        #train_nertagger(training_texts, new_model_dir)
        
        # Set up the new trained nertagger and defining layers to be removed later on
        tagger = NerTagger(model_dir = new_model_dir)
        
        # Tag the files using the new nertagger
        testing_files = [key for key, value in files.items() if int(value) == testing]
        tag_files(new_model_dir, testing_files, tagger, use_vabamorfcorpustagger)
            
    # Get results of model
    extract_results_to_txt_file(model_directory, files)
    
    print(f"Mudel {model_directory} treenitud.")

To train the model the `model_directory` must contain a `settings.py` file

In [11]:
'''
models = [
    'model_default_with_vabamorftagger',
    'model_local_features_without_morph',
    'model_morph_without_lemmas',
    'model_morph_with_lemmas',
    'model_morph_with_lemmas_and_sentences',
    'model_morph_with_lemmas_and_sentences_and_gazzetteer',
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_initial'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer1and2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc_variants')    
]
'''
models = [
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer1and2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc_variants')
]
for model in models:
    train_model(model)

(!) Märgendan
Märgendatud fail V6ru_R2pina_Kahkva_id24674_1868a.json
Märgendatud fail L22ne_Martna_Martna_id14205_1869a.json
Märgendatud fail Harju_Juuru_Juuru_id19451_1886a.json
Märgendatud fail Tartu_Kodavere_Ranna_id11316_1845a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22448_1881a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24908_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id20382_1901a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id13162_1866a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20418_1888a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24521_1888a.json
Märgendatud fail P2rnu_Mihkli_Mihkli_id1099_1852a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16184_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id10343_1871a.json
Märgendatud fail L22ne_Ridala_V6nnu_id2373_1889a.json
Märgendatud fail Harju_Hageri_Kohila_id2634_1882a.json
Märgendatud fail L22ne_Martna_Martna_id10803_1890a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13294_1887a.json
Märgendatud fail P2rnu_Tori_Sindi_id377

Märgendatud fail Tartu_V6nnu_Ahja_id21768_1867a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20314_1888a.json
Märgendatud fail Tartu_R6ngu_Aakre_id4282_1888a.json
Märgendatud fail J2rva_Tyri_S2revere_id6796_1883a.json
Märgendatud fail V6ru_R2pina_R2pina_id10711_1868a.json
Märgendatud fail Harju_Kose_Palvere_id23525_1887a.json
Märgendatud fail Tartu_R6ngu_Aakre_id13836_1829a.json
Märgendatud fail P2rnu_Halliste_Penuja_id657_1885a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id11390_1880a.json
Märgendatud fail V6ru_Vastseliina_Misso_id11633_1886a.json
Märgendatud fail Viljandi_P6ltsamaa_Adavere_id20850_1896a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id3008_1884a.json
Märgendatud fail J2rva_Ambla_Ambla_id7255_1886a.json
Märgendatud fail Tartu_Torma_Avinurme_id23576_1872a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id7375_1869a.json
Märgendatud fail Tartu_Kodavere_Pala_id18163_1862a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24037_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22714_

Märgendatud fail L22ne_Kullamaa_Sooniste_id3541_1880a.json
Märgendatud fail J2rva_Tyri_Tyri-Alliku_id2315_1897a.json
Märgendatud fail J2rva_Tyri_S2revere_id11683_1874a.json
Märgendatud fail Saare_Kaarma_Loona_id7575_1899a.json
Märgendatud fail V6ru_P6lva_K2hri_id21590_1851a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16351_1884a.json
Märgendatud fail Tartu_V6nnu_Ahja_id11361_1872a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16121_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21444_1866a.json
Märgendatud fail J2rva_Tyri_S2revere_id14702_1887a.json
Märgendatud fail L22ne_Martna_Martna_id12705_1885a.json
Märgendatud fail Tartu_Torma_Avinurme_id6291_1861a.json
Märgendatud fail Harju_Kose_Palvere_id16297_1881a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id18115_1877a.json
Märgendatud fail Tartu_Kambja_Vana-Prangli_id19091_1909a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14900_1882a.json
Märgendatud fail Tartu_V6nnu_Ahja_id19074_1887a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id15463_188

Märgendatud fail Harju_Kose_Kose-Uuem6isa_id4916_1868a.json
Märgendatud fail Harju_Juuru_Juuru_id20528_1870a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13655_1882a.json
Märgendatud fail Tartu_Torma_Avinurme_id20455_1871a.json
Märgendatud fail L22ne_Martna_Martna_id12611_1884a.json
Märgendatud fail Tartu_Maarja-Magdaleena_J6e_id15191_1864a.json
Märgendatud fail P2rnu_Halliste_Pornuse_id4791_1869a.json
Märgendatud fail Tartu_R6ngu_Aakre_id8042_1827a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13144_1876a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17542_1885a.json
Märgendatud fail Tartu_Kodavere_Pala_id17298_1857a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id1642_1884a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id14538_1866a.json
Märgendatud fail Tartu_Torma_Avinurme_id24645_1823a.json
Märgendatud fail Harju_Keila_Keila_id11680_1886a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17059_1884a.json
Märgendatud fail Harju_Hageri_Kohila_id10480_1870a.json
Märgendatud fail V6ru_Kanepi_Krootuse_id2451

Märgendatud fail V6ru_Urvaste_Urvaste_id8023_1859a.json
Märgendatud fail Viljandi_Paistu_Holstre_id9107_1836a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21575_1867a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id5382_1863a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id7573_1879a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id6169_1879a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24526_1888a.json
Märgendatud fail J2rva_Tyri_S2revere_id14656_1887a.json
Märgendatud fail Tartu_Maarja-Magdaleena_J6e_id12945_1877a.json
Märgendatud fail J2rva_Peetri_Silmsi_id22492_1867a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23150_1872a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id4473_1883a.json
Märgendatud fail Tartu_Kodavere_Ranna_id15127_1864a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14086_1882a.json
Märgendatud fail Saare_Kihelkonna_Pidula_id5682_1890a.json
Märgendatud fail Tartu_Kodavere_Pala_id22067_1871a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23206_1872a.json
Märgendatud fail Tart

...

Märgendatud fail P2rnu_Halliste_Penuja_id701_1885a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id19110_1867a.json
Märgendatud fail J2rva_Tyri_V22tsa_id20630_1902a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id19258_1870a.json
Märgendatud fail J2rva_Tyri_S2revere_id8113_1885a.json
Märgendatud fail Tartu_Kambja_Vana-Prangli_id17547_1906a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16450_1884a.json
Märgendatud fail V6ru_R2pina_R2pina_id11883_1866a.json
Märgendatud fail J2rva_J2rva-Jaani_Karinu_id1197_1866a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id11540_1866a.json
Märgendatud fail Harju_Kuusalu_K6nnu_id14416_1868a.json
Märgendatud fail Viljandi_P6ltsamaa_Adavere_id19020_1892a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id3227_1884a.json
Märgendatud fail Tartu_Kodavere_Pala_id17430_1859a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14781_1882a.json
Märgendatud fail Harju_Kose_Palvere_id18876_1883a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18783_1897a.json
Märgendatud fail Tartu_V6nn

Märgendatud fail Tartu_V6nnu_Ahja_id18352_1886a.json
Märgendatud fail L22ne_Kullamaa_Kuij6e_id15473_1876a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id1309_1884a.json
Märgendatud fail L22ne_Reigi_K6rgessaare_id23290_1894a.json
Märgendatud fail J2rva_Tyri_Kirna_id23810_1874a.json
Märgendatud fail Viljandi_Paistu_Holstre_id10692_1905a.json
Märgendatud fail Tartu_Kodavere_Pala_id21377_1869a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id18057_1877a.json
Märgendatud fail Harju_Kose_Palvere_id18877_1883a.json
Märgendatud fail Harju_Jyri_Rae_id277_1874a.json
Märgendatud fail V6ru_R2pina_R2pina_id10845_1868a.json
Märgendatud fail Tartu_V6nnu_Ahja_id9071_1871a.json
Märgendatud fail Tartu_Kursi_Puurmani_id20904_1874a.json
Märgendatud fail J2rva_Tyri_V22tsa_id22259_1913a.json
Märgendatud fail J2rva_Tyri_Kirna_id23935_1875a.json
Märgendatud fail J2rva_Tyri_Kirna_id24596_1880a.json
Märgendatud fail J2rva_Tyri_V22tsa_id22155_1911a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id15364_1876a

Märgendatud fail Viljandi_P6ltsamaa_Pajusi_id5261_1872a.json
Märgendatud fail L22ne_Martna_Martna_id12491_1883a.json
Märgendatud fail Harju_Kose_Triigi_id12117_1871a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23208_1872a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17090_1884a.json
Märgendatud fail Harju_Kose_Palvere_id18191_1882a.json
Märgendatud fail J2rva_J2rva-Jaani_Karinu_id1193_1866a.json
Märgendatud fail V6ru_Kanepi_Krootuse_id16254_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14796_1882a.json
Märgendatud fail V6ru_R2pina_R2pina_id1168_1863a.json
Märgendatud fail V6ru_R6uge_Saaluse_id9577_1878a.json
Märgendatud fail V6ru_R2pina_Kahkva_id5750_1887a.json
Märgendatud fail J2rva_Tyri_Tyri-Alliku_id1370_1894a.json
Märgendatud fail V6ru_R2pina_Kahkva_id19205_1866a.json
Märgendatud fail J2rva_Tyri_S2revere_id13737_1884a.json
Märgendatud fail Saare_Kihelkonna_Kotlandi_id20759_1867a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id22767_1869a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_

Märgendatud fail Tartu_Kodavere_Pala_id24085_1873a.json
Märgendatud fail Tartu_Torma_Avinurme_id14737_1905a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id13504_1899a.json
Märgendatud fail L22ne_Hanila_Massu_id446_1913a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18591_1895a.json
Märgendatud fail Harju_Hageri_Kohila_id21266_1885a.json
Märgendatud fail Tartu_R6ngu_Aakre_id5921_1890a.json
Failid märgendatud 185.9109070301056 sekundiga.
(!) Märgendan
Märgendatud fail V6ru_R6uge_Saaluse_id8753_1877a.json
Märgendatud fail Viljandi_Helme_Leebiku_id14149_1888a.json
Märgendatud fail L22ne_Martna_Martna_id14159_1884a.json
Märgendatud fail V6ru_Vastseliina_Misso_id24907_1886a.json
Märgendatud fail V6ru_R2pina_R2pina_id10213_1870a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16362_1885a.json
Märgendatud fail J2rva_Tyri_S2revere_id5550_1881a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id10860_1880a.json
Märgendatud fail Harju_Juuru_Kaiu_id16280_1884a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_i

Märgendatud fail P2rnu_Audru_V6lla_id5156_1877a.json
Märgendatud fail Viljandi_Paistu_Holstre_id10727_1905a.json
Märgendatud fail J2rva_Ambla_Ambla_id6086_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17663_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id11306_1872a.json
Märgendatud fail Tartu_V6nnu_Ahja_id11374_1872a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id18022_1877a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id18055_1877a.json
Märgendatud fail Harju_Juuru_Kaiu_id1087_1909a.json
Märgendatud fail Harju_Hageri_Kohila_id2480_1881a.json
Märgendatud fail V6ru_R6uge_Saaluse_id8830_1877a.json
Märgendatud fail Tartu_Torma_Avinurme_id21473_1872a.json
Märgendatud fail L22ne_Kullamaa_Sooniste_id2085_1872a.json
Märgendatud fail Harju_Kuusalu_Kolga_id11652_1887a.json
Märgendatud fail J2rva_Ambla_Ambla_id11857_1882a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id7180_1884a.json
Märgendatud fail J2rva_Tyri_S2revere_id8880_1886a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21240_